In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers, optimizers
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
train

In [ ]:
def append_ext(fn):
    return fn+".jpg"

In [ ]:
train['image_name'] = train['image_name'].apply(append_ext)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=0)

In [ ]:
for train_index, test_index in sss.split(train, train.target):
...     print("TRAIN:", train_index, "TEST:", test_index)
...     train, valid = train.loc[train_index], train.loc[test_index]

In [ ]:
train

In [ ]:
valid

In [ ]:
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
test

In [ ]:
test['image_name'] = test['image_name'].apply(append_ext)
test

In [ ]:
train.target = train.target.astype('str')
train.target.dtype

In [ ]:
valid.target = valid.target.astype('str')
valid.target.dtype

In [ ]:
datagen=ImageDataGenerator(rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

In [ ]:
train_generator=datagen.flow_from_dataframe(
dataframe=train,
directory="../input/siim-isic-melanoma-classification/jpeg/train/",
x_col="image_name",
y_col="target",
subset="training",
batch_size=32,
seed=42,
shuffle=False,
class_mode="sparse",
target_size=(256, 256))

In [ ]:
valid_generator=datagen.flow_from_dataframe(
dataframe=valid,
directory="../input/siim-isic-melanoma-classification/jpeg/train/",
x_col="image_name",
y_col="target",
subset="training",
batch_size=32,
seed=42,
shuffle=False,
class_mode="sparse",
target_size=(256, 256))

In [ ]:
test_datagen=ImageDataGenerator(rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

In [ ]:
test_generator=test_datagen.flow_from_dataframe(
dataframe=test,
directory="../input/siim-isic-melanoma-classification/jpeg/test/",
x_col="image_name",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(256, 256))

In [ ]:
pip install efficientnet

In [ ]:
import efficientnet.tfkeras as efn

In [ ]:
model2 = tf.keras.Sequential([
        efn.EfficientNetB0(
            input_shape=(256,256, 3),
            weights='imagenet',
            include_top=False
        ),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

In [ ]:
history = model2.fit_generator(train_generator,
            validation_data = valid_generator,
            epochs = 3,
            verbose = 1)

In [ ]:
y_pred = model2.predict(test_generator)

In [ ]:
y_pred_2 = np.argmax(y_pred, axis = 1)

In [ ]:
y_pred_2

In [ ]:
y_pred[1]

In [ ]:
y_pred_final = y_pred[:, 1]
y_pred_final

In [ ]:
test_dataset = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')
test_dataset

In [ ]:
test_dataset.image_name.values

In [ ]:
pred_df = pd.DataFrame(columns = ['image_name', 'target'])
pred_df['image_name'] = test_dataset.image_name.values
pred_df['target'] = y_pred_2
pred_df

In [ ]:
pred_df.to_csv('efficient_net.csv', index = False)